### Лабораторная работа №4

#### <span style="color:blue">Задание.</span> Провести классификацию найденного датасета, методами наивного Байеса  . В формате *Markdown* написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

Сначала мы импортируем необходимые библиотеки и нормализуем данные:

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score

data = pd.read_csv("adult.data")
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 
                'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 
                'hours-per-week', 'native-country', 'income']
data.columns = column_names

data = data.replace(' ?', pd.NA)

data = data.dropna()
data = pd.get_dummies(data, columns=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'])
data['income'] = data['income'].map({' <=50K': 0, ' >50K': 1})

Разделение данных на обучающий и тестовый наборы

In [17]:
X = data.drop('income', axis=1)
y = data['income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Используем модель BernoulliNB, которая хорошо подходит для бинарных признаков.

In [18]:
model = BernoulliNB()  

Задаем сетки параметров и ищем лучшие:

In [19]:
param_grid = {'alpha': [0.1, 0.5, 1.0, 1.5, 2.0], 'binarize': [0.0, 0.5, 1.0, 1.5, 2.0]}

grid_search = GridSearchCV(model, param_grid, cv=5)

grid_search.fit(X_train, y_train)

print("Лучшие параметры: ", grid_search.best_params_)

Лучшие параметры:  {'alpha': 0.1, 'binarize': 1.0}


Обучаем модель на обучающей выборке:

In [20]:
best_model = grid_search.best_estimator_

best_model.fit(X_train, y_train)

BernoulliNB(alpha=0.1, binarize=1.0)

Выбор гиперпараметра alpha (гиперпараметр сглаживания) зависит от характера данных. Значение по умолчанию alpha=1.0 является стандартным выбором.

Оценка производительности модели

In [21]:
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Точность: {accuracy}')
print('Отчет о классификации:\n', report)

Точность: 0.775070445880988
Отчет о классификации:
               precision    recall  f1-score   support

           0       0.79      0.96      0.87      4535
           1       0.64      0.21      0.32      1498

    accuracy                           0.78      6033
   macro avg       0.71      0.59      0.59      6033
weighted avg       0.75      0.78      0.73      6033



Для повышения надежности оценки производительности модели, проведем кросс-валидацию(например, с 5 фолдами).

In [22]:
cv_scores = cross_val_score(best_model, X, y, cv=5)

print('Результаты кросс-валидации:', cv_scores)
print(f'Средняя точность: {cv_scores.mean()}')

Результаты кросс-валидации: [0.77374441 0.76856764 0.77635942 0.76873342 0.77635942]
Средняя точность: 0.7727528599334956


Мы успешно провели классификацию датасета с использованием методов наивного Байеса, выбрав модель BernoulliNB для бинарной классификации. Мы также оценили производительность модели на тестовом наборе и провели кросс-валидацию для более надежной оценки её обобщающей способности.